## Process text and select only relevant sections based on keywords 

In [4]:
import pandas as pd
import pickle
import numpy as np
import pyodbc

In [5]:
with open('us_all_data.p', 'rb') as f:
    us_results = pickle.load(f)

with open('uk_all_data.p', 'rb') as f:
    uk_results = pickle.load(f)

In [6]:
def clean_text(text):
    '''
    to read xml text and select only relevant text for analyses
    input: xml file
    output: full relevant text  
    '''
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(text, 'xml')
    for tag in soup.body.find_all(['fig']):
        tag.decompose()
    sections = soup.find_all('sec')
    sections = [sec.get_text() for sec in sections]
    #To exclude the 1st page of the report
    sections = [sec for sec in sections if len(re.findall(r'telephone', sec, flags = re.IGNORECASE))==0 ]
    relevance = sections[:3]
    
    return '\n'.join(relevance)

In [ ]:
keywords = ['\\beconomic outlook\\b','\\brecent economic development\\b',
            '\\bkey issues\\b','\\bbackdrop\\b','\\bstaff appraisal\\b',
            '\\boverview\\b','\\boutlook\\b','\\bintroduction\\b']

import re

country =[]
year = []
published_date = []
content = []
for i in range(len(us_results)):
    if len(re.findall(r'|'.join(keywords), us_results[i].Content, flags = re.IGNORECASE ))>0:
        country.append(us_results[i].Country)
        year.append(us_results[i].ProjectedYear)
        published_date.append(us_results[i].PublicationDate)
        text = clean_text(us_results[i].Content)
        content.append(text)

for i in range(len(uk_results)):
    if len(re.findall(r'|'.join(keywords), uk_results[i].Content, flags = re.IGNORECASE ))>0:
        country.append(uk_results[i].Country)
        year.append(uk_results[i].ProjectedYear)
        published_date.append(uk_results[i].PublicationDate)
        text = clean_text(uk_results[i].Content)
        content.append(text)

In [ ]:
sample_data = dict({'Country': country,
                 'ProjectedYear': year,
                 'PublishedDate': published_date,
                 'Content': content})

In [ ]:
pickle.dump(sample_data, open('sample_data_1.p',"wb"))